I found that using the following specific format for same as links pointing to Wikidata items or properties will work best. This notebook runs back through and corrects any cases where I've gone the wrong way. This notebook fixes that problem. I also used it as an opportunity to look at some new ways of working with the wikibaseintegrator package that I think may work better in some cases. I need to revisit this in terms of how to invoke the fast run mode that I've been struggling to figure out.

In [1]:
import os
import requests
from wikibaseintegrator import (WikibaseIntegrator, wbi_login)
from wikibaseintegrator.datatypes import URL
from wikibaseintegrator.models import Claims
from wikibaseintegrator.entities import ItemEntity, PropertyEntity
from wikibaseintegrator.wbi_config import config as wbi_config
from wikibaseintegrator.wbi_enums import ActionIfExists

In [2]:
bot_name = 'GEOKB_CLOUD'

wbi_config['MEDIAWIKI_API_URL'] = os.environ[f'WB_API_{bot_name}']
wbi_config['SPARQL_ENDPOINT_URL'] = os.environ[f'WB_SPARQL_{bot_name}']
wbi_config['WIKIBASE_URL'] = os.environ[f'WB_URL_{bot_name}']
wbi_config['USER_AGENT'] = 'Update wikidata same as links'

login_instance = wbi_login.Login(
    user=os.environ[f'WB_BOT_{bot_name}'],
    password=os.environ[f'WB_BOT_PASS_{bot_name}']
)

wbi = WikibaseIntegrator(login=login_instance, is_bot=True)


In [6]:
query_wd_same_as = """
PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>

SELECT ?item ?same_as
WHERE {
  ?item wdt:P84 ?same_as .
  FILTER CONTAINS(STR(?same_as), "https://www.wikidata.org/")
}
"""

r = requests.get(
    'https://geokb.wikibase.cloud/query/sparql',
    params={'query': query_wd_same_as, 'format': 'json'},
)

identifiers = [i['item']['value'].split('/')[-1] for i in r.json()['results']['bindings']]

identifiers[:5]

[]

In [4]:
def update_wd_same_as(id):
    if id.startswith('P'):
        entity = PropertyEntity(api=wbi).get(id)
    else:
        entity = ItemEntity(api=wbi).get(id)

    claim_urls = []
    for claim in entity.claims.get('P84'):
        claim_urls.append(claim.mainsnak.datavalue['value'].replace('https://', 'http://'))

    new_claims = Claims()
    for url in claim_urls:
        new_claims.add(
            URL(
                prop_nr='P84',
                value=url
            )
        )

    entity.claims.add(new_claims, action_if_exists=ActionIfExists.REPLACE_ALL)
    response = entity.write(
        summary='Update wikidata same as links',
    )
    return response.id

In [5]:
for id in identifiers:
    print(update_wd_same_as(id))

Q26280
Q159583
Q158209
Q158208
Q158206
Q158205
Q158711
Q158210
P101
P10
P1
P102
Q158204
Q55213
Q158710
Q158715
Q158716
Q158736
Q158713
Q152412
Q158748
Q158712
P113
P107
P146
P108
P133
P149
P121
P148
P111
P37
P144
P106
P129
P110
P122
P139
P130
P145
P109
P115
P38
P62
Q138353
Q138354
Q138355
Q138356
Q138357
Q138360
Q138362
Q138411
Q138412
Q138358
Q138359
Q138410
Q138361
Q138413
Q152320
Q152616
Q158131
Q152354
Q158123
Q138414
Q158129
Q158128
Q158130
Q158127
Q158118
Q158135
Q158155
Q158136
Q158152
Q158139
Q158146
Q158138
Q158148
Q158154
Q41263
Q158218
Q158215
Q158721
Q41257
Q159582
Q223
Q159576
Q3645
Q159581
Q158702
Q26282
Q25371
Q158708
Q158214
Q159577
Q158707
Q159579
Q25370
Q44202
Q25372
Q158220
Q159569
Q158720
Q158699
Q18
Q25356
Q158705
Q158191
Q158697
Q25357
Q44201
Q41259
Q41258
Q158714
Q159046
Q158165
Q158217
Q158704
Q19
Q158167
Q159574
Q41260
Q158709
Q159575
Q407
Q159578
Q158698
Q25373
Q25368
Q158719
Q44200
Q25366
Q44145
Q158703
Q25365
Q25360
Q158187
Q158219
Q159572
Q158160
Q158161
Q2